### Tokenize Cancer Dependency Map data in order to send it into the Geneformer embedding extractor
- create an Anndata object with `.X` as the raw gene expression information, and `.obs` containing patient ID, cell line, and disease (as well as disease state -- cancerous or not)
- converts from gene name to ensembl ID

Part 2:
- obtain highly variable genes (top 500), and restrict the Anndata object to these 500 genes
- tokenize it using the Geneformer tokenizer and save the resulting dataset to disk

### Data processing

In [1]:
# Imports

import sys
from pathlib import Path
import pandas as pd 
import numpy as np
import scanpy as sc
import anndata as ad
from scipy.sparse import csr_matrix
# import scvelo as scv
import h5py
import os
import scipy.sparse as sp
from datasets import load_from_disk
# import pooch
import matplotlib.pyplot as plt
from matplotlib import rcParams
import h5py
from tqdm import tqdm
import pickle
import torch

#change to path to your Geneformer directory
sys.path.append('/work/magroup/kaileyhu/Geneformer')
from geneformer import EmbExtractor
from geneformer import TranscriptomeTokenizer

print("imports done")

2025-02-23 23:19:09.517816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered


imports done


In [2]:
# Check torch device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Found device {device}")

Found device cuda:0


In [3]:
#files
metadata = pd.read_csv("/work/magroup/kaileyhu/datasets/depmap/metadata.csv")
omics_expr = pd.read_csv("/work/magroup/kaileyhu/datasets/depmap/OmicsExpressionProteinCodingGenesTPMLogp1.csv")

metadata.set_index('ModelID', inplace = True)
omics_expr.set_index("Unnamed: 0", inplace = True)

In [4]:
# protein coding genes

adata = ad.AnnData(omics_expr)
adata.obs_names = [str(i).split(" ")[0] for i in omics_expr.index]
adata.var_names = [str(i).split(" ")[0] for i in omics_expr.columns]

adata.obs["patient_id"] = omics_expr.index
adata.obs["cell_line"] = metadata["OncotreeLineage"]
adata.obs["disease"] = metadata["OncotreePrimaryDisease"]

for patient in adata.obs_names:
    adata.obs.loc[patient, 'cell_line'] = metadata.loc[patient, "StrippedCellLineName"]
    if (metadata.loc[patient, "OncotreePrimaryDisease"] != "Non-Cancerous"):
        adata.obs.loc[patient, 'disease_state'] = "Cancerous"
    else:
        adata.obs.loc[patient, 'disease_state'] = "Non-Cancerous"
    adata.obs.loc[patient, 'disease'] = metadata.loc[patient, "OncotreePrimaryDisease"]

In [5]:
# import ensembl_df of gene -> ensembl id matrix

ensembl_path = "/work/magroup/kaileyhu/Geneformer/geneformer/ensembl_mapping_dict_gc95M.pkl"

def invert_dict(dict_obj):
    return {v: k for k, v in dict_obj.items()}

with open(ensembl_path, "rb") as f:
    id_gene_dict = pickle.load(f)
    gene_id_dict = invert_dict(id_gene_dict)

In [6]:
lst = []
genes = []

for gene in adata.var_names:
    gene2 = gene.split(" ")[0]
    if gene2 in id_gene_dict:
        lst.append(id_gene_dict[gene2])
        genes.append(gene2)
    else:
        lst.append(None)


filtered_results = []

res = []
for val in lst:
    if val is not None:
        filtered_results.append(val)

adata2 = adata[:,genes]
adata2.var_names = filtered_results 
adata2.var['ensembl_id'] = filtered_results
adata2.obs['n_counts'] = adata2.X.sum(axis=1)
adata2.obs["cell_type"] = adata.obs['cell_line']
adata2.obs["disease"] = adata.obs['disease']
adata2.obs["disease_state"] = adata.obs['disease_state']
adata2.obs["batch"] = 0
adata2.obs["patient_id"] = adata.obs["patient_id"]
adata2.var['gene_name'] = genes

In [7]:
adata2

AnnData object with n_obs × n_vars = 1479 × 18858
    obs: 'patient_id', 'cell_line', 'disease', 'disease_state', 'n_counts', 'cell_type', 'batch'
    var: 'ensembl_id', 'gene_name'

In [9]:
adata2.write_h5ad("/work/magroup/kaileyhu/datasets/depmap/processed/omics_expr.h5ad",compression='gzip')

### Obtain highly variable genes

In [ ]:
sc.pp.highly_variable_genes(adata2, n_top_genes=500, inplace=True)

In [ ]:
adata_hvg = adata2[:, adata2.var["highly_variable"]]

In [ ]:
adata_hvg.write_h5ad("/work/magroup/kaileyhu/datasets/depmap/processed/hvg/omics_expr_hvg_500.h5ad",compression='gzip')

### Tokenization

In [ ]:
tk = TranscriptomeTokenizer({"cell_type": "cell_type", "disease": "disease", "disease_state": "disease_state", "patient_id" : "patient_id"}, 
                            nproc=16,
                            special_token = True, #true for 95M
                            model_input_size=4096)  

# 18130 total genes

In [ ]:
tk.tokenize_data('/work/magroup/kaileyhu/datasets/depmap/processed/hvg/', 
                 "/work/magroup/kaileyhu/res/", 
                 "hvg_500_tokenized", 
                 file_format="h5ad")

### Tokenization (non-HVG)

In [ ]:
tk = TranscriptomeTokenizer({"cell_type": "cell_type", "disease": "disease", "disease_state": "disease_state", "patient_id" : "patient_id"}, 
                            nproc=16,
                            special_token = True, #true for 95M
                            model_input_size=4096)  

# 18130 total genes

In [ ]:
tk.tokenize_data('/work/magroup/kaileyhu/datasets/depmap/processed/', 
                 "/work/magroup/kaileyhu/res/", 
                 "all_genes_tokenized", 
                 file_format="h5ad")

### Embeddings

In [ ]:
os.listdir("../../../Geneformer/fine_tuned_models/")

In [ ]:
extractor_type = "cancer"
path_to_Geneformer = "../../../Geneformer"

if extractor_type == "cancer":
    model = f"{path_to_Geneformer}/gf-12L-95M-i4096_CLcancer"
    n_classes = 71
elif extractor_type == "cardio":
    model = f"{path_to_Geneformer}/fine_tuned_models/gf-6L-30M-i2048_CellClassifier_cardiomyopathies_220224"
    n_classes = 3
else: # pretrained
    model = f"{path_to_Geneformer}/gf-12L-30M-i2048"
    n_classes = 0

In [ ]:
# initiate EmbExtractor
print(f"Starting emb extractor...")
embex = EmbExtractor(model_type="CellClassifier",
                     num_classes=n_classes,
                     emb_layer=-1,
                     emb_label=["cell_type","disease", "disease_state"],
                     labels_to_plot=["disease_state", "disease"],
                     forward_batch_size=200,
                     nproc=16)

In [ ]:
# extracts embedding from input data
print(f"Extracting embeddings from {extractor_type}...")
embs = embex.extract_embs(model,
                          "/work/magroup/kaileyhu/res/hvg_500_tokenized.dataset",
                          "/work/magroup/kaileyhu/res/",
                          f"omics_expr_original_embs_{extractor_type}")

In [ ]:
embex.plot_embs(embs=embs, 
                plot_style="umap",
                output_directory="/work/magroup/kaileyhu/res/",  
                output_prefix="emb_plot")